In [ ]:
! pip install -Uq einops

In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from einops import rearrange
from joblib import Parallel, delayed

In [ ]:
base_dir = '/kaggle/input/seti-breakthrough-listen/train/'
train_df = pd.read_csv('/kaggle/input/seti-breakthrough-listen/train_labels.csv')
train_df['path'] = [base_dir + id[0] + '/' + id + '.npy' for id in train_df['id']]

In [ ]:
import matplotlib.pyplot as plt
from einops import rearrange, reduce

file = train_df['path'][5]

fig, ax = plt.subplots(figsize=(20,20))
x = np.load(file).astype(float)
ax.imshow(rearrange(x, 't0 t f -> f (t0 t)'))
plt.xlabel('Time')
plt.ylabel('Frequency')
plt.show();

In [ ]:
def sigmoid(x): return 1/(1 + np.exp(-x))

fig, ax = plt.subplots(figsize=(20,20))
x = sigmoid(np.load(file).astype(float))
ax.imshow(rearrange(x, 't0 t f -> f (t0 t)'))
plt.xlabel('Time')
plt.ylabel('Frequency')
plt.show();

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
x = sigmoid(np.load(file).astype(float))
ax.imshow(rearrange(x[[0,2,4,1,3,5]], '(t01 t02) t f -> (t01 f) (t02 t)', t01 = 2))
plt.xlabel('Time')
plt.ylabel('Frequency')
plt.show();

In [ ]:
import numpy as np
def sigmoid(x): return 1/(1 + np.exp(-x))

import os
import cv2
from pathlib import Path
from einops import rearrange
def npy2jpg(file, return_np = False):
    
    x = np.load(file).astype(float)
    x = sigmoid(x)
    x = rearrange(x[[0,2,4,1,3,5]], '(t01 t02) t f -> (t01 f) (t02 t)', t01 = 2)
    x = x * 255
    
    img_path = '/kaggle/working' / Path(*Path(file).parts[-3:]).with_suffix('.png')
    os.makedirs(img_path.parent, exist_ok=True)
    cv2.imwrite(str(img_path), x)
    
    if return_np:
        return x

In [ ]:
o = Parallel(n_jobs=4)(delayed(npy2jpg)(path) for path in tqdm(train_df['path']))

In [ ]:
from zipfile import ZipFile
import os
import shutil

# iterate over all the files in directory
for folderName, subfolders, filenames in os.walk('/kaggle/working/train'):
    if folderName == '/kaggle/working/train':
        continue
    # create a ZipFile object
    #print(folderName)
    with ZipFile(folderName.split('/')[-1] + '.zip', 'w') as zipObj:
        for filename in filenames:
            # create complete filepath of file in directory
            filePath = os.path.join(folderName, filename)
            # add file to zip
            zipObj.write(filePath, os.path.basename(filePath))
            # delete the file to open space
            os.remove(filePath)
shutil.rmtree("/kaggle/working/train")